# DataFrame

http://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html#pyspark-sql-module

In [31]:
import pyspark
import os

from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'ipython3'

## Tworzenie DataFrame'u

### Plik

In [32]:
data_path = './SparkSQLdata/'

In [33]:
# json
people = spark.read.json(data_path+'people.json')

In [34]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [35]:
# csv
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')

In [36]:
people_txt.show()

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



### Kolekcja Row'ów

In [37]:
newPerson1 = Row(name='Greg', age=32)

In [38]:
newPerson1

Row(age=32, name='Greg')

In [39]:
newPerson1.name

'Greg'

In [40]:
newPerson1.age

32

In [41]:
newPerson1['age']

32

In [42]:
'age' in newPerson1

True

In [43]:
newPerson = Row("age", "name")

In [44]:
newPerson2 = newPerson(24, 'Alice')

In [45]:
newPerson2

Row(age=24, name='Alice')

In [46]:
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [47]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, 
                                     newPerson5, newPerson6, newPerson7])

In [48]:
newPeopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



### Inne lokalne kolekcje

Typy danych: http://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html#module-pyspark.sql.types

Kilka podstawowych: IntegerType, DoubleType, FloatType, StringType, BooleanType, NullType

In [49]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

In [50]:
# definicja schematu
# StructType ~ Row
schema = StructType([StructField("V1", IntegerType()), StructField("V2", StringType())])

In [51]:
# lokalna kolekcja - lista list
df = spark.createDataFrame([[1,2],[3,4]], schema)

In [52]:
df.show()

+---+---+
| V1| V2|
+---+---+
|  1|  2|
|  3|  4|
+---+---+



### RDD

Przechodzenie RDD <-> DF

In [53]:
type(people)

pyspark.sql.dataframe.DataFrame

In [54]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [55]:
people.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None)]

DF -> RDD

In [56]:
type(people.rdd)

pyspark.rdd.RDD

In [57]:
people.rdd.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None)]

In [58]:
people.rdd.map(tuple).collect()

[(None, 'Michael'),
 (30, 'Andy'),
 (19, 'Justin'),
 (35, 'Emma'),
 (None, None),
 (31, None)]

RDD -> DF

In [59]:
people.rdd.toDF().show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [60]:
people.rdd.map(tuple).toDF().show()

+----+-------+
|  _1|     _2|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [61]:
sc = spark.sparkContext

In [62]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF().show()

+---+---+
| _1| _2|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
+---+---+



In [75]:
# do toDF można podać schemat
schema = StructType([StructField("A", IntegerType()), StructField("B", StringType())])
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).show()

+---+---+
|  A|  B|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
+---+---+



In [76]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).collect()[0]

Row(A=0, B='1')

> **TODO**: Na dwa sposoby stwórz DataFrame z 3 wierszami i 3 kolumnami - dwie typu string, jedna numeryczna

In [84]:
schema1 = StructType([StructField("S1", StringType()), StructField("S2", StringType()), StructField("I1", IntegerType())])
sc.parallelize([(x, x+1, x+2) for x in range(3)]).toDF(schema1).show()

+---+---+---+
| S1| S2| I1|
+---+---+---+
|  0|  1|  2|
|  1|  2|  3|
|  2|  3|  4|
+---+---+---+



In [94]:
constr = Row("s1","s2","i1")
r1 = constr("a","b",2)
r2 = constr("c","D",3)
r3 = constr("E","f",5)
spark.createDataFrame([r1,r2,r3]).show()

+---+---+---+
| s1| s2| i1|
+---+---+---+
|  a|  b|  2|
|  c|  D|  3|
|  E|  f|  5|
+---+---+---+



> **TODO**: Z podanego RDD utwórz DataFrame z nazwanymi kolumnami `name` i `age` oraz odpowiednimi typami (string i int)

In [95]:
myRDD = sc.textFile(data_path+"people.txt")
myRDD.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [101]:
myRDD.map(lambda x: x.split(", ")).collect()
schema2 = StructType([StructField("Name", StringType()), StructField("Age", IntegerType())])

In [104]:
## wszystko w stringach
#myRDD.map(lambda x: x.split(", ")).toDF().printSchema()

myRDD.map(lambda x: x.split(", ")).map(lambda x: (x[0], int(x[1]))).toDF(schema2).printSchema()


root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)



****

## Praca z DataFrame'ami

### Ogolne wiadomosci na temat danych
printSchema, show, columns, dtypes <br>
Znane z RDD np: count, take, head

In [66]:
people.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [67]:
people_txt.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)



In [68]:
people_txt.show(1)

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
+-------+----+
only showing top 1 row



In [105]:
people.show(1)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
+----+-------+
only showing top 1 row



Liczba wierszy

In [106]:
people.count()

6

Lista kolumn

In [107]:
people.columns

['age', 'name']

Lista kolumn wraz z typami danych

In [108]:
newPeopleDF.dtypes

[('age', 'bigint'), ('name', 'string')]

### Odwolania do poszczegolnych kolumn

In [109]:
people.age

Column<b'age'>

In [110]:
people['age']

Column<b'age'>

In [111]:
people[0]

Column<b'age'>

### Dodanie/usuniecie kolumny
withColumn, drop

In [112]:
people.withColumn(colName = 'ageNextYear', col = people.age +1).show()

+----+-------+-----------+
| age|   name|ageNextYear|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
|  35|   Emma|         36|
|null|   null|       null|
|  31|   null|         32|
+----+-------+-----------+



In [113]:
people.drop('age').show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|   Emma|
|   null|
|   null|
+-------+



### Zmiany nazwy kolumny

In [115]:
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')

In [116]:
people_txt.show()

+-------+----+
|   name| age|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



### Podstawowe statystyki kolumn w DataFrame'ie.
describe

In [117]:
people.describe().show()

+-------+-----------------+-------+
|summary|              age|   name|
+-------+-----------------+-------+
|  count|                4|      4|
|   mean|            28.75|   null|
| stddev|6.849574196011505|   null|
|    min|               19|   Andy|
|    max|               35|Michael|
+-------+-----------------+-------+



In [118]:
people.describe('age').show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|                4|
|   mean|            28.75|
| stddev|6.849574196011505|
|    min|               19|
|    max|               35|
+-------+-----------------+



### Braki danych
isNull, isNotNull<br>
fillna, dropna, replace

In [119]:
newPeopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



In [120]:
newPeopleDF.filter(newPeopleDF.age.isNull()).show()

+----+-----+
| age| name|
+----+-----+
|null| null|
|null|Peter|
+----+-----+



In [121]:
newPeopleDF.filter(newPeopleDF.age.isNotNull()).show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| 33| null|
| 32|Peter|
| 40| Greg|
+---+-----+



In [122]:
newPeopleDF.fillna(-1).show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| -1| null|
| 33| null|
| -1|Peter|
| 32|Peter|
| 40| Greg|
+---+-----+



In [123]:
newPeopleDF.fillna({'age':-1, 'name':'unknown'}).show()

+---+-------+
|age|   name|
+---+-------+
| 32|   Greg|
| 24|  Alice|
| -1|unknown|
| 33|unknown|
| -1|  Peter|
| 32|  Peter|
| 40|   Greg|
+---+-------+



In [124]:
newPeopleDF.fillna({'name':'unknown'}).replace('unknown', 'NN').show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null|   NN|
|  33|   NN|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



In [125]:
newPeopleDF.dropna().show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| 32|Peter|
| 40| Greg|
+---+-----+



In [126]:
newPeopleDF.dropna(subset='age').show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| 33| null|
| 32|Peter|
| 40| Greg|
+---+-----+



### Funkcje wprost ze skladni SQL 
select, where (alias filter), orderBy

In [127]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [128]:
people.\
select('name', 'age').\
where(people.name.like('%n%')).\
orderBy(people.age.asc()).\
show()
# select: wyswietl kolumny 'name' i 'age'
# where: uwzglednij tylko imiona (name) zaweirajace litere 'n'
# orderBy: zbior posortuj rosnaco po kolumnie 'age'

+------+---+
|  name|age|
+------+---+
|Justin| 19|
|  Andy| 30|
+------+---+



In [129]:
# gdy chcemy zobaczyć wszystkie kolumny select jest zbędny
people.\
where(people.name.like('%n%')).\
orderBy(people.age.asc()).\
show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
| 30|  Andy|
+---+------+



> **TODO**: Wyswietl imiona ludzi ze zbioru `people` starszych niz 29 lat. Wyniki posortuj alfabetycznie po kolumnie name

### Operacje na zbiorach
union - dziala jak UNION ALL w SQL. <br>
intersect (INTERSECT z SQLa), subtract (EXCEPT z SQLa)

In [130]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [131]:
people_txt.show()

+-------+----+
|   name| age|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



In [132]:
people.union(people_txt).show()

+-------+-------+
|    age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
|     35|   Emma|
|   null|   null|
|     31|   null|
|Michael|   29.0|
|   Andy|   30.0|
| Justin|   19.0|
+-------+-------+



In [133]:
people.union(people_txt.select(people.columns)).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
|35.0|   Emma|
|null|   null|
|31.0|   null|
|29.0|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



###############################################################################################################

# Zadania 1

****
Przygotowanie danych

In [134]:
data_path = './SparkSQLdata/'

In [135]:
people = spark.read.json(data_path+'people.json')
employees = spark.read.json(data_path+'employees.json')
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')

In [136]:
newPerson1 = Row(name=u'Greg', age=32)
newPerson = Row("age", "name")
newPerson2 = newPerson(24, 'Alice')
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [137]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, 
                                     newPerson5, newPerson6, newPerson7])

In [138]:
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')

In [139]:
allPeople = spark.read.parquet(data_path+'allPeople.parquet')

****

1. Wyswietl imiona ludzi ze zbioru `people`, o ktorych wielku nie mamy informacji.

In [141]:
people.where(people.age.isNull()).select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   null|
+-------+



2. Na koniec zbioru `people` doklej zbiór `newPeopleDF` oraz `people_txt`. Tak utworzony DataFrame nazwij `allPeople`

3. Wybierz ze zbioru `allPeople` te wiersze, ktore wystepuja rowniez w zbiorze `people`.

4. Wybierz ze zbioru `allPeople` te imiona, ktore NIE wystepuja w zbiorze `people_txt`.

5. Usun ze zbioru `allPeople` wiersze, ktore zawieraja same braki danych.

6. Do zbioru `allPeople` dodaj kolumne 'age' zawierajaca wiek powiekszony o 1. Zmien nazwe oryginalnej kolumny 'age' na 'starting_age'.

7. Ile unikatowych rekordow znajduje sie w zbiorze `allPeople`?

8. **⋆** Do zbioru `employees` dodaj kolumny: z pensjami po 0-10% podwyżce (dla każdego losowo z rozkładu jednostajnego) oraz z różnicą pomiędzy pensją przed i po podwyżce. Zbiór posortuj alfabetycznie.

In [ ]:
from pyspark.sql import functions as f

###############################################################################################################

## Praca z DataFrame'ami

In [ ]:
data_path = './SparkSQLdata/'
people = spark.read.json(data_path+'people.json')
employees = spark.read.json(data_path+'employees.json')
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')
newPeople = spark.read.parquet(data_path+'newPeople.parquet')
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')

### Funkcje SparkSQL

In [ ]:
from pyspark.sql import functions as f

In [ ]:
people.select(f.min("age")).show()

In [ ]:
people.select(f.min("age").alias("min_age")).show()

In [ ]:
people.withColumn("random", f.randn(42)).show()

In [ ]:
people.withColumn("random", f.exp("age")).show()

### join
inner (domyslny)

In [ ]:
people.join(other=employees, on='name', how='inner').show()

<b> Uwaga ogólna </b><br>
Join to stosunkowo popularna, ale kosztowna operacja.<br>
W sytuacji, kiedy jeden z łaczonych DataFramow jest znacznie mniejszy (w szczegolnosci na tyle mały, że w całości mieści się w pamięci), zaleca sie zastosowanie <i>broadcast hash join</i>.<br>
(Mała tabela zostanie zebrana do pamięci i wysłana do każdego noda).<br>
W niektórych przypadkach optymalizator sam za nas zdecyduje o zastosowaniu <i>broadcast hash join</i>.

In [ ]:
from pyspark.sql.functions import broadcast
newPeople.join(broadcast(spark.createDataFrame([Row(age=20, name='Greg')])), on='name').show()

> **TODO**: Spośród osób (`people`, `newPeople`, `people_txt`), dla których mamy informacje o zarobkach (zbiór `employees`). Ile zarabia najmłodsza osoba?

> **TODO**: Dla każdego pracownika (`employees`), dla którego mamy informacje o wieku (`people`, `newPeople`, `people_txt`). Dodaj do pensji 0.1% za każdy rok życia. Oblicz koszt takiego 'bonusu urodzinowego' dla pracodawcy. 

### groupBy

In [ ]:
newPeople.groupBy()

 Przez GroupedData mamy dostep do takich funkcji jak:<br>
 avg, max, min, sum, count, agg <br>
 (dla wygody, do funkcji 'agg' mamy tez dostep bezposrednio na DataFrame)

In [ ]:
newPeople.groupBy().max().show()

In [ ]:
newPeople.groupBy('name').count().show()

In [ ]:
newPeople.groupBy('name').agg(f.min('age').alias('min_age'), f.max('age').alias('max_age'),\
                              f.count('name').alias('n_people')).show()

In [ ]:
newPeople.agg(f.min('age'), f.max('age'), f.count('name')).show()

> **TODO**: Ile jest unikatowych (występujących tylko 1 raz) imion w połączonych zbiorach `people`, `newPeople` oraz `people_txt`?

> **TODO**: Ile lat mają osoby, których imiona występują tylko raz w połączonych zbiorach `people`, `newPeople` oraz `people_txt`?

****

#### Nowy DataFrame

In [ ]:
import random

In [ ]:
random.seed(123)

years = 10
names = ['Alice', 'Betty', 'Chris', 'Dan', 'Greg']
unique_names_count = len(names)
names = sorted(names*years)
year = [y for y in range(2000, 2000+years)]*len(names)
starting_salary = [round(random.gauss(4000, 1000),2) for i in range(unique_names_count)]
salary = [0 for i in range(years*unique_names_count)]
salary[::years] = starting_salary
for n in range(unique_names_count):
    for y in range(years-1):
        index = (years*n+1)+y
        salary[index] = round(salary[index-1]*(1+random.gauss(0.1,0.09)),2)

In [ ]:
salaryHistory = spark.createDataFrame([Row(name=n, year=y, salary=s) for n,y,s in zip(names, year, salary)])
salaryHistory = salaryHistory.filter((salaryHistory['name'] != 'Greg') | (salaryHistory['year'] != 2006))
salaryHistory = salaryHistory.union(spark.createDataFrame([Row('Alice', 3000, 2000)]))

In [ ]:
salaryHistory.show()

****

> **TODO**: Ile razy powtarza się każde z imion w `salaryHistory`?

> **TODO**: Na podstawie `salaryHistory` stworz tabelę zależności średniej, minimalnej i maksymalnej pensji od roku. Posortuj lata malejąco. Pensje podaj z dokładnością do pełnych wartości.

### Window functions
**over**

Służy do obliczania agregowanych wartości w grupach definiowanych oknem (window).<br>
Zwraca wiele rekordow (tyle ile na wejsciu w grupie).

partitionBy - definiuje podział danych na okna<br>
orderBy - definiuje sortowanie wewnątrz każdego z okien<br>
Frame (rangeBetween/rowsBetween) - definiuje offset<br>

In [ ]:
from pyspark.sql.window import Window

**partitionBy**

In [ ]:
allPeople = spark.read.parquet(data_path+'allPeople.parquet')

In [ ]:
# definicja 'okna'
myWindowSpec = Window.partitionBy('name')

In [ ]:
# wywołanie funkcji na kazdym 'oknie'
allPeople.withColumn('nameCount', \
                     f.count(allPeople['name']).over(myWindowSpec)).show()

> **TODO**: Do zbioru `salaryHistory` dodaj kolumnę 'avgSalaryDiff', która będzie zawierała różnicę pomiedzy pensją z danego roku, a średnią pensją osoby na przestrzeni wszytskich lat.

**partitionBy + orderBy**

In [ ]:
# przykład: rank
# - musimy zdefiniować dodatkowo sortowanie wewnątrz każdej z grup
# - zwraca lp dla kolejnych rekordów posortowanych według zadanych kolumn
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year'])
ranked = f.rank().over(windowSpec)
salaryHistory.withColumn('ranked', ranked).show()

In [ ]:
salaryHistory_tmp = salaryHistory.filter(salaryHistory.name.isin('Alice', 'Greg'))

In [ ]:
# rank, dense_rank, percent_rank, row_number
windowSpec = Window.partitionBy('name').orderBy('year')
ranked = (f.rank()).over(windowSpec)
dense_rank = (f.dense_rank()).over(windowSpec)
percent_rank = (f.percent_rank()).over(windowSpec)
row_number = (f.row_number()).over(windowSpec)
salaryHistory_tmp.withColumn('ranked', ranked).withColumn('dense_rank', dense_rank).\
withColumn('percent_rank', percent_rank).withColumn('row_number', row_number).\
show()

Inne warte uwagi funkcje: <br>
ntile, cume_dist, first, lag, lead

> **TODO**: Dla zbioru `salaryHistory`, porównaj pensje ludzi pomiedzy najwcześniejszym i najpóźniejszym rokiem ich pracy.

**partitionBy + orderBy + rangeBetween/rowsBetween**

In [ ]:
# przykład: średnia ruchoma
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year'])\
.rowsBetween(-1,1)
movingAvg = (f.avg(salaryHistory['salary'])).over(windowSpec)
salaryHistory.withColumn('movingAvg', movingAvg).show()

In [ ]:
# przykład: średnia ze wszystkich rekordów do aktualnego włącznie
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year']).\
rowsBetween(Window.unboundedPreceding,Window.currentRow)
movingAvg = (f.avg(salaryHistory['salary'])).over(windowSpec)
salaryHistory.withColumn('movingAvg', movingAvg).show()

In [ ]:
salaryHistory.withColumn('movingAvg', movingAvg).filter("name == 'Alice'").show()

In [ ]:
# podobny efekt uzyskamy poniższym zapytaniemm. 
# Różnica: rekordy w jednej grupie (imię, rok) nie zostaną rozdzielone 
import sys
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year']).\
rangeBetween(-sys.maxsize,0)
movingAvg = (f.avg(salaryHistory['salary'])).over(windowSpec)
salaryHistory.withColumn('movingAvg', movingAvg).show()

In [ ]:
salaryHistory.withColumn('movingAvg', movingAvg).filter("name == 'Alice'").show()

###############################################################################################################

# Zadania 2

1. Ile lat mają osoby, których imiona występują tylko raz w połączonych zbiorach `people`, `newPeople` oraz `people_txt`? Rozwiązując problem zastosuj window functions.

2. Czy komukolwiek obniżyła się pensja w stosunku do roku poprzedniego? <br>
a. Ile osób było kiedykolwiek w takiej sytuacji?<br>
b. Jaki jest rozkład częstości takich przypadków w zależności od roku?

3. Oblicz różnicę w pensjach w stosuku do <br>
a. najwyższej pensji danej osoby<br> 
b. drugiej najwyższej pensji danej osoby<br>

###############################################################################################################

### SQL

Spark wspiera ANSI SQL 2003 (SQL3)

Aby użyć zbioru danych w zapytaniu SQL musimy go najpierw zarejestrować.

In [ ]:
salaryHistory.createOrReplaceTempView('salaryHistory')
# salaryHistory.registerTempTable('salaryHistorySQL') - wersja dla sparka 1.x
# view NIE jest 'persisted in memory', to nadal nie jest akcja

In [ ]:
spark.sql('select * from salaryHistory limit 2').show()

In [ ]:
type(spark.sql('select * from salaryHistory limit 2'))

In [ ]:
spark.sql('select name, avg(salary) avg_sal from salaryHistory group by name').show()

In [ ]:
spark.sql('select name, salary, avg(salary) over (partition by name) avg_sal from salaryHistory').show()

In [ ]:
spark.catalog.dropTempView("salaryHistory")

### UDFs
Poza funkcjami dostępnymi w module pyspark.sql.functions, można też tworzyć własne (User Defined Functions) i używać ich w zapytaniach SparkSQL <br>
<i>Uwaga:</i> Po UDFy sięgamy w ostateczności - optymalizacja <br>
<i>Uwaga:</i> UDF napisany w Pythonie (lub R) bedzie mial gorszy performance niż UDF napisany w Scali lub Javie. Ale możemy nasz UDF napisać w Scali/Javie i zarejestrować do użycia w Pythonie. <br>
Blogpost o tym jak w pySparku używać UDF napisanej w scali: <br>
https://medium.com/@ingwbaa/using-scala-udfs-in-pyspark-b70033dd69b9 <br>
Blogpost o ulepszeniach w UDF (pandas UDF) dla pySparka od Sparku 2.3: <br>
https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html

In [ ]:
from pyspark.sql.types import IntegerType, StringType, FloatType

In [ ]:
def udfPower3(value):
    return(value**3)

In [ ]:
udfPower3(3.14)

Aby stosować zdefiniowaną funkcję, musimy ja zarejestrować. (Driver roześle funkcje do wszystkich egzekutorów)

In [ ]:
power3 = f.udf(udfPower3, FloatType())

In [ ]:
salaryHistory.select(power3(salaryHistory.salary)).show()

### Spark wspiera Hive

Spark SQL wspiera HiveQL. <br>
Spark SQL wspiera rownież wczytawanie/zapisywanie danych z/do Apache Hive.<br>

Wiecej informacji na temat integracji z Hive:<br>
https://spark.apache.org/docs/latest/sql-programming-guide.html#compatibility-with-apache-hive <br>
https://spark.apache.org/docs/latest/sql-programming-guide.html#hive-tables